masking을 어떤 것을 하느냐가 성능에 영향을 줄 것.

개별 token을 그냥 masking하는 것이 오히려 좋은 성능을 보일 수도.

아니면 keyword 추출하는데 영향을 미친 token을 masking하는 방식?

bounding box 단위? token 단위? masing 어떻게 할까

tokenizer 다르게 씀으로 인해 주요 단어들이 UNK로 튕겨나갈 수 있다.

--> tokenizer를 바꾸고 새로 학습?

--> tokenizer t5 그냥 쓰고 뒤에 한국어 tokenizer붙여 vocab 크기를 늘리는 방식?

tokenizer의 학습: vocab의 빈도를 학습. 빈도가 크면 그냥 사용. 빈도가 작으면 vocab을 잘라서 사용.

In [7]:

!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import random
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

In [6]:
from transformers import T5Tokenizer, T5TokenizerFast, PreTrainedTokenizer, PreTrainedTokenizerBase

import re
import sentencepiece as spm

# The special tokens of T5Tokenizer is hard-coded with <extra_id_{}>
# Created another class UDOPTokenizer extending it to add special visual tokens like <loc_{}>, etc.

#class UdopTokenizer(T5Tokenizer):
class UdopTokenizer(AutoTokenizer):

    def __init__(
        self,
        vocab_file,
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        extra_ids=100,
        loc_extra_ids=501,
        other_extra_ids=200,
        additional_special_tokens=[],
        sp_model_kwargs=None,
        **kwargs
    ):
        # Add extra_ids to the special token list
        if extra_ids > 0 and not "<extra_id_0>" in additional_special_tokens:
            additional_special_tokens = ["<extra_id_{}>".format(i) for i in range(extra_ids)]
            additional_special_tokens.extend(["<extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_l_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["<extra_t_id_{}>".format(i) for i in range(extra_ids)])
            additional_special_tokens.extend(["</extra_t_id_{}>".format(i) for i in range(extra_ids)])

        if loc_extra_ids > 0 and not "<loc_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<loc_{}>".format(i) for i in range(loc_extra_ids)])

        if other_extra_ids > 0 and not "<other_0>" in additional_special_tokens:
            additional_special_tokens.extend(["<other_{}>".format(i) for i in range(other_extra_ids)])
        print(additional_special_tokens)
        PreTrainedTokenizer.__init__(
            self,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            extra_ids=extra_ids,
            additional_special_tokens=additional_special_tokens,
            **kwargs,
        )

        self.sp_model_kwargs = {} if sp_model_kwargs is None else sp_model_kwargs

        self.vocab_file = vocab_file
        self._extra_ids = extra_ids
        self._loc_extra_ids = loc_extra_ids
        self._other_extra_ids = other_extra_ids

        self.sp_model = spm.SentencePieceProcessor(**self.sp_model_kwargs)
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids + self._other_extra_ids

    def get_vocab(self):
        vocab = {self.convert_ids_to_tokens(
            i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        if token.startswith("<extra_id_"):
            match = re.match(r"<extra_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4
        elif token.startswith("<extra_l_id_"):
            match = re.match(r"<extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3
        elif token.startswith("</extra_l_id_"):
            match = re.match(r"</extra_l_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2
        elif token.startswith("<extra_t_id_"):
            match = re.match(r"<extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids - self._extra_ids
        elif token.startswith("</extra_t_id_"):
            match = re.match(r"</extra_t_id_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids - self._loc_extra_ids
        elif token.startswith("<loc_"):
            match = re.match(r"<loc_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1 - self._other_extra_ids
        elif token.startswith("<other_"):
            match = re.match(r"<other_(\d+)>", token)
            num = int(match.group(1))
            return self.vocab_size - num - 1

        return self.sp_model.piece_to_id(token)

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        if index < self.sp_model.get_piece_size():
            token = self.sp_model.IdToPiece(index)
        else:

            if index > self.sp_model.get_piece_size() + self._extra_ids * 5 + self._loc_extra_ids - 1:
                index_loc = self.vocab_size - 1 - index
                token = f"<other_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 5 - 1:
                index_loc = self.vocab_size - self._other_extra_ids - 1 - index
                token = f"<loc_{index_loc}>"
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 4 - 1:
                token = "</extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 3 - 1:
                token = "<extra_t_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids * 2 - 1:
                token = "</extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 2 - 1 - index)
            elif index > self.sp_model.get_piece_size() + self._extra_ids - 1:
                token = "<extra_l_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 3 - 1 - index)
            elif index > self.sp_model.get_piece_size() - 1:
                token = "<extra_id_{}>".format(self.vocab_size - self._other_extra_ids - self._loc_extra_ids - self._extra_ids * 4 - 1 - index)
            else:
                raise
        return token



In [7]:

"""
    codes for just preprocessing
"""

def normalText(t):
    if type(t) is float:
        if t == int(t):
            t = int(t)
    t = str(t)
    return t.strip()


def get_prop(node, name):
    title = node.get("title")
    props = title.split(";")
    for prop in props:
        (key, args) = prop.split(None, 1)
        args = args.strip('"')
        if key == name:
            return args
    return None


def get_bb(bb):
    bbs = [float(j) for j in bb]
    xs, ys = [], []
    for i, b in enumerate(bbs):
        if i % 2 == 0:
            xs.append(b)
        else:
            ys.append(b)
    return [min(xs), min(ys), max(xs), max(ys)]


from PIL import Image
import torchvision.transforms as T
from torchvision.transforms import functional as F

"""
    codes for just preprocessing image data
"""
def get_visual_bbox(image_size=224):
    image_feature_pool_shape = [image_size//16, image_size//16]
    visual_bbox_x = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[1] + 1),
        1.0,
    ) / image_feature_pool_shape[1])
    visual_bbox_y = (torch.arange(
        0,
        1.0 * (image_feature_pool_shape[0] + 1),
        1.0,
    ) / image_feature_pool_shape[0])
    visual_bbox_input = torch.stack(
        [
            visual_bbox_x[:-1].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[:-1].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
            visual_bbox_x[1:].repeat(
                image_feature_pool_shape[0], 1),
            visual_bbox_y[1:].repeat(
                image_feature_pool_shape[1], 1).transpose(
                    0, 1),
        ],
        dim=-1,
    ).view(-1, 4)
    return visual_bbox_input

class Normalize(object):
    def __init__(self, mean, std, format='rgb'):
        self.mean = mean
        self.std = std
        self.format = format.lower()

    def __call__(self, image):
        if 'bgr' in self.format:
            image = image[[2, 1, 0]]
        if '255' in self.format:
            image = image * 255
        if image.size(0) == 1:
            image = image.repeat(3, 1, 1)
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image

def img_trans_torchvision(image, image_size=224):
    trans = T.Compose([
            T.Resize([image_size,image_size]),
            T.ToTensor(),
            Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

    image = trans(image)  # copy to make it writeable
    return image

In [8]:
class DataCollatorForT5DocCLS:
    """
    Data collator used for T5 document classification
    """
    def __init__(self, json_data=None, tokenizer=None, meta_path=None, input_length=None, target_length=None, pad_token_id=None, decoder_start_token_id=None):

        self.json_data = json_data
        self.tokenizer = tokenizer #이전에 만든 udop tokenizer를 불러옴
        self.input_length = input_length
        self.target_length = target_length
        self.pad_token_id = pad_token_id
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, user_prompt ,ori_input_ids, ori_bbox_list, labels=None):

        # "원래 input text 정보 & bounding box"
        # -->
        # "prompt text 정보 + 원래 input text 정보" list
        # +
        # [0,0,0,0]을 promt text token 개수만큼 + 원래 bounding box

        #prompt_text = 'document classification.'
        prompt_text = user_prompt
        prompt_ids =  self.tokenizer.encode(prompt_text, add_special_tokens=False)
        input_ids = prompt_ids + ori_input_ids
        bbox_list = [[0,0,0,0]] * len(prompt_ids) + ori_bbox_list

        if(labels!=None):  #label은 classification에서만 수행
        #인줄 알았는데 layout modeling 이런것도 다 output이 있으니까 label==output 인건가..???
          labels = self.tokenizer.encode(labels, add_special_tokens=True)

        return input_ids, labels, bbox_list


# 해당 부분은 json파일의 문장을 line-by-line으로 읽는 것으로 해당 함수 수정 완료.
def read_ocr_core_engine(json_data,image_path, file_idx, tokenizer, max_seq_length=None, num_img_embeds=None, image_size=224):
    #max_seq_length와 num_img_embeds 는 원본 코드에서도 안쓰는데 왜있는거지?

    #file_ 와 image_dir 모두 1 2 3 ... index 임.

    data = json_data[file_idx] # 하나로 합쳐진 json파일에서 현재 idx 가져옴.

    rets = []
    n_split = 0

    #page_size = (1280,720)
    page_size = data['form'][0]['sheet_size']

    tiff_images =  Image.open(f"{image_path}/image_{file_idx}.png")

    image = img_trans_torchvision(tiff_images, image_size)

    text_list, bbox_list = [], []
    for form in data['form']: #문장별로 쪼갬
      for word in form['words']: #단어별로 쪼갬

        if word == ' ': #띄어쓰기는 건너뛰기
          continue

        sub_tokens = tokenizer.tokenize(word['text']) #단어별로 쪼갠걸 다시 토큰화 (하나의 단어도 여러개의 토큰 가능)
        for sub_token in sub_tokens:
          text_list.append(sub_token)
          bbox_list.append(word['box']) #현재는 단어별 bbox, 추후 문장별 bbox로도 수정 가능
          #bbox_list.append(form['box'])

    if len(text_list) > 0:
      rets.append([text_list, bbox_list, image, page_size])

    assert len(text_list) == len(bbox_list)
    n_split = len(rets)

    return rets, n_split


import pandas as pd
from torch.utils.data import Dataset
from tqdm import tqdm


EMPTY_BOX = [0, 0, 0, 0]
SEP_BOX = [1000, 1000, 1000, 1000]

class RvlCdipDataset(Dataset):

    #NUM_LABELS = 16

    def __init__(self , xml_sample_loc , json_sum_loc, image_path, tokenizer , data_args , mode='train'):

        """ Structure of data directory:

            --- xml_sample_loc (.csv)
                   ├── images_url
                   └── labels_url
            --- data (folder)
                   └── processed_sample{index} .json
        """
        self.main_df = pd.read_csv(xml_sample_loc) # xml_sample.csv 파일 저장
        self.image_path = image_path

        with open(json_sum_loc, 'r', encoding='utf8') as f:
            self.main_json_data = json.load(f)

        if mode == 'train': #train ,val, test 에 따라 사용하는 data의 범위가 다름. (근데 self-supervised도 이거 필요 있나..? )
            file_data_range = ( 0 , int(len(self.main_df) * 0.6 ) )
        elif mode == 'val':
            file_data_range = ( int(len(self.main_df) * 0.6 ) , int(len(self.main_df) * 0.8 ) )
        elif mode == 'test':
            file_data_range = ( int(len(self.main_df) * 0.8 ) , int(len(self.main_df) ) )
        else:
            raise NotImplementedError

        self.cls_bbox = EMPTY_BOX[:]
        self.pad_bbox = EMPTY_BOX[:]
        self.sep_bbox = SEP_BOX[:]

        self.tokenizer = tokenizer
        self.max_seq_length = data_args.max_seq_length
        self.num_img_embeds = 0

        label_list = ['a','b'] #get_rvlcdip_labels() #classification task에서만 사용하는 변수
        self.label_list = label_list
        self.label_map = dict(zip(list(range(len(self.label_list))), self.label_list))
        self.n_classes = len(label_list)
        self.label_list = label_list

        self.image_size = data_args.image_size

        self.examples = []
        self.labels = []
        self.images = []

        self.cls_collator = DataCollatorForT5DocCLS( #기존에 정의한 토크나이저 선언
                json_data = self.main_json_data ,
                  tokenizer=tokenizer,
            )

        results = [self.load_file(file_idx) for file_idx in tqdm(range(file_data_range[0],file_data_range[1]))]
        for labels, examples, images in results:
            self.labels += labels
            self.examples += examples
            self.images += images
        assert len(self.labels) == len(self.examples)

    def load_file(self, file_idx):

        labels = []
        examples = []
        images = []

        labels.append(0) ############### label 미정으로 일단 다 0 ##########
        examples.append(file_idx)
        images.append(file_idx)

        return labels, examples, images

    def __getitem__(self, index): #완료
        try:
            label = self.labels[index]
            label = self.label_map[int(label)]

            rets, n_split = read_ocr_core_engine(json_data = self.main_json_data , image_path = self.image_path , file_idx = index , tokenizer = self.tokenizer, max_seq_length = self.max_seq_length, num_img_embeds = self.num_img_embeds, image_size = self.image_size)

            if n_split == 0:
                # Something wrong with the .ocr.json file
                print(f"EMPTY ENTRY in index {index}")
                return self[(index + 1) % len(self)]
            for i in range(n_split): #정상적으로 코드 실행됬다면 n_split==1 임.
                text_list, bbox_list, image, page_size = rets[i]
                (width, height) = page_size
                bbox = [  #이미지 크기에 맞게 정규화
                    [
                        b[0] / width,
                        b[1] / height,
                        b[2] / width,
                        b[3] / height,
                    ]
                    for b in bbox_list
                ]

                visual_bbox_input = get_visual_bbox(self.image_size) # (x_min, y_min, x_max, y_max) 형태의 좌표로 이루어진 텐서 반환

                input_ids = self.tokenizer.convert_tokens_to_ids(text_list) #토큰 자른것들을 token id들로 변환

                input_ids, labels, bbox_input = self.cls_collator("user prompt", input_ids, bbox, label) #prompt 붙여서 최종 input,bbox,label을 만듦. ################################
                attention_mask = [1] * len(input_ids)
                decoder_attention_mask = [1] * len(labels)

                char_list = [0]
                char_bbox_list = [[0,0,0,0]]
                char_ids = torch.tensor(char_list, dtype=torch.long)
                char_bbox_input = torch.tensor(char_bbox_list, dtype=torch.float)

                bbox_input = torch.tensor(bbox_input, dtype=torch.float)
                labels = torch.tensor(labels, dtype=torch.long)
                input_ids = torch.tensor(input_ids, dtype=torch.long)
                attention_mask = torch.tensor(attention_mask, dtype=torch.long)
                decoder_attention_mask = torch.tensor(decoder_attention_mask, dtype=torch.long)
                assert len(bbox_input) == len(input_ids)
                assert len(bbox_input.size()) == 2
                assert len(char_bbox_input.size()) == 2

                return_dict =  {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "labels": labels,
                    "seg_data": bbox_input,
                    "visual_seg_data": visual_bbox_input,
                    "decoder_attention_mask": decoder_attention_mask,
                    "image": image,
                    'char_ids': char_ids,
                    'char_seg_data': char_bbox_input
                }
                assert input_ids is not None

                return return_dict
        except: #오류가 났다는 거는 파일이 없다는 것. 해당 상황에서는 index+1 파일 불러오는 것으로 대체
            #image는 로딩 중 오류 생긴 파일 그냥 해당 index가 없게 저장해서 문제 없음.
            #json파일도 오류 생긴건 해당 index없어서 걸러짐.
            print(f"{index} 파일을 {0}로 대체")

            return self.__getitem__(0)

            #return self[(index + 1) % len(self)]

    #def get_labels(self): # classification에서 label의 종류 출력하는 함수. 우리는 필요 없을 듯.
    #    return list(map(str, list(range(self.NUM_LABELS))))

    def pad_tokens(self, input_ids, bbox): #이건 그냥 길이 max_len에 맞게 맞추는 함수
        # [CLS], sentence, [SEP]
        tokenized_tokens = self.tokenizer.build_inputs_with_special_tokens(input_ids)
        start_token, _, end_token = tokenized_tokens[0], tokenized_tokens[1:-1], tokenized_tokens[-1]

        sentence = tokenized_tokens
        expected_seq_length = self.max_seq_length - self.num_img_embeds
        mask = torch.zeros(expected_seq_length)
        mask[:len(sentence)] = 1

        bbox = [self.cls_bbox] + bbox + [self.sep_bbox]
        while len(sentence) < expected_seq_length:
            sentence.append(self.tokenizer.pad_token_id)
            bbox.append(self.pad_bbox)

        assert len(sentence) == len(bbox)
        return (sentence, mask, bbox, start_token, end_token)

#test

In [9]:
import json

xml_sample_loc = '/content/drive/MyDrive/ColabNotebooks/산학_테스트/xml_sample_20230519.csv'
json_sum_loc = '/content/drive/MyDrive/ColabNotebooks/산학_테스트/sumall_processed_sample.json'
image_path = '/content/drive/MyDrive/ColabNotebooks/산학_테스트/image' #folder 경로임.

In [10]:
import argparse

# data_args를 Namespace 객체로 변환합니다.
data_args = argparse.Namespace(
    max_seq_length=500,
    image_size=224
)
#json 하나로 묶어서 pikkle로 대체
my_tokenizer = UdopTokenizer.from_pretrained("/content/drive/MyDrive/ColabNotebooks/산학_테스트/tokenizer_finetuned_ket5_by_xml_data")

dataset = RvlCdipDataset(xml_sample_loc , json_sum_loc , image_path,  my_tokenizer , data_args , mode='train')

100%|██████████| 1844/1844 [00:00<00:00, 342801.90it/s]


In [11]:
for i in range(1844): #1800개 9분 걸림. drvie에서 불러오는 상황. local에서 불러오면 더 빠를 것.
  x = dataset.__getitem__(i)

4 파일을 0로 대체
8 파일을 0로 대체
13 파일을 0로 대체
44 파일을 0로 대체
80 파일을 0로 대체
82 파일을 0로 대체
84 파일을 0로 대체
86 파일을 0로 대체
382 파일을 0로 대체
486 파일을 0로 대체
767 파일을 0로 대체
932 파일을 0로 대체
979 파일을 0로 대체
980 파일을 0로 대체
1013 파일을 0로 대체
1086 파일을 0로 대체
1088 파일을 0로 대체
1153 파일을 0로 대체
1154 파일을 0로 대체
1155 파일을 0로 대체
1156 파일을 0로 대체
1157 파일을 0로 대체
1158 파일을 0로 대체
1198 파일을 0로 대체
1215 파일을 0로 대체
1230 파일을 0로 대체
1293 파일을 0로 대체
1320 파일을 0로 대체
1456 파일을 0로 대체
1543 파일을 0로 대체
1551 파일을 0로 대체
1552 파일을 0로 대체
1808 파일을 0로 대체
